# Enhancing the Word Completion Model: String Slices and Frequencies

You can now write a simple program for finding all possible word completions using either `str.startswith` or a custom function `startswith` that uses a while loop to compare letters one-by-one.
This unit shows you one more general purpose alternative to `str.starswith`, and then we will add frequency information to get only the most likely word completion.

## Cutting strings into slices

### Use for word prediction

In the previous unit, we described a specific way of manually checking whether `long_string` is a possible completion of `short_string`.
The procedure broke down into the following steps:

1. Determine the length of `short_string`.
1. Use this to compute the positions 1, 2, 3, ..., n where `short_string` and `long_string` must be identical.
1. Check for each position that identity holds.

But instead we could have also done the following.

1. Determine the length of `short_string`.
1. Truncate `long_string` so that it has the same length as `short_string`.
   That is to say, cut off the part at the end that makes `long_string` longer than `short_string.
1. Test if `short_string` and the truncated version of `long_string` are identical.
   If so, `long_string` is a possible completion of `short_string.
   
Let's illustrate this idea with an example first.
Is `yesty` a possible completion of `yes`?
The answer is obviously yes, but here are the specific steps that derive this conclusion:

1. The length of `yes` is 3.
1. If we truncate `yesty` to a length of 3, we get `yes`.
1. Since `yes == yes` is `True`, we conclude that `yesty` is a possible completion of `yes`.

Python makes it very easy to truncate strings to a specific length.
The key tool for this is **slices**.
The specification of a slice is similar to how we specify positions in a string.
Instead of writing, say, `"yesty"[0]` to get the first letter of the string `"yesty"`, we write `"yesty[0:3]"` to get the substring that spans from position 0 to position 3.
So the general syntax is `"string"[start_position:end_position]`.

In [ ]:
# getting the substring between positions 0 and 3
print("yesty"[0:3])

In [ ]:
# truncating yester to the length of yes
print("yesty"[0:len("yes")])

Note that the start position of a slice does not need to be 0, we can also pick other positions.

In [ ]:
print("yesty"[1:3])

In the special case where the start position is 0, it can be completely omitted.

In [ ]:
print("yesty"[0:3])
print("yesty"[:3])

You may remember this notation from earlier units when we used commands like `hamlet[:100]` to look at the first 100 word tokens of *Hamlet*.

**Exercise.**
For each one of the following commands, add a comment that explains what it does.
If a command raises an error, comment it out and explain what causes the error.

In [ ]:
print("yesty"[:len("yes")])
# your comment:

print("yesty"[:len("yesty")])
# your comment:

print("yesty"[2:5])
# your comment:

print("yesty"[2:])
# your comment:

print("yesty"[:])
# your comment:

print("yesty"[])
# your comment:

print("yesty"[4:2])
# your comment:

With slices, there is yet another way of implementing the `complete_word` function.

**Exercise.**
The implementation of `complete_word` below now uses slices.
Add a comment to each line that explains what this line does.
Pay particular attention to how the slice is defined.

In [ ]:
def complete_word(string, dictionary):
    completions = []
    for word in dictionary:
        if word[:len(string)] == string:
            list.append(completions, word)
    return completions

As evidence that this version of `complete_word` computes the same output as previous implementations, you can look at the output for the commands below and compare it to the output we got in the previous unit.
Make sure you run the first cell, though, so that `dictionary` is defined.

In [ ]:
import urllib.request
import re

def read_file(filename):
    with open(filename, "r", encoding="utf-8") as text:
        return text.read()

# download the file
url = "https://raw.githubusercontent.com/dwyl/english-words/master/words.txt"
urllib.request.urlretrieve(url, "words.txt")
dict_string = read_file("words.txt")

# tokenize dict_string;
# remember that each word is on its own line, so [^\n]+ does the trick
dictionary = re.findall(r"[^\n]+", dict_string)

In [ ]:
complete_word("excite", dictionary)

In [ ]:
complete_word("yes", dictionary)

### An important note on slices and positions

There is one thing about slices that might be a bit confusing, and that's how the end position is interpreted.
You might think that something like `"yesty"[0:3]` means "print the characters at positions 0, 1, 2, and 3".
But that's not what we get.

In [ ]:
print("Let's compare positions to slices:")
teststring = "yesty"
for pos in [0,1,2,3]:
    print("At position", pos, "of", teststring, "we have the character", teststring[pos])
    
for pos in [0,1,2,3]:
    print("But the slice of", teststring, "from 0 to", pos, "is", teststring[0:pos])

In order to understand what's going on here, we have to think a bit more carefully about how Python handle's positions in a string.
As far as Python is concerned, a string looks similar to the following thing:

In [ ]:
print(list(enumerate("yesty")))

This format makes it very clear how positions correspond to specific characters in the string.
A pair `(p, "c")` means that at position `p` in the string we see character `c`.
So far so good, this is exactly what we expect given what we have learned about positions so far.
But why, then, does `"yesty"[0:3]` give us `yes` rather than `yest`?

Without going too much into technical details, this is because slices have a specific interpretation.
Suppose that we look at the string `"yesty"` as a list of pairs as in the output above.
Then the command `"yesty"[0:3]` tells Python to start at position `0` and keep moving right until it sees the number `3`.
The characters it has seen along the way are combined into a string.
Here is how this logic works for `"yesty"[0:3]`:

1. We start at position 0.
1. We take one step to the right and see a `y`.
1. We take another step to the right and see the position 1.
   We don't care about this position, so we move on.
1. We take another step to the right and see an `e`.
   We add this to the `y` that we saw before, yielding `ye`.
1. We take another step to the right and see the position 2.
   We do not care about this position either, so we move on.
1. We take another step to the right and see an `s`.
   We add that to the already built string `ye` to get `yes`.
1. We take another step to the right and see the position 3.
   This is where we have to stop, so we call it a day and return the string built so far: `yes`.

Looking at slicing from this perspective also explains a few other things that are pretty confusing at first.
For example, consider what happens when we give Python a position that does not exist in the string.

In [ ]:
print("yesty"[len("yesty")])

Since `"yesty"` has a length of 5, there are 5 positions: 0, 1, 2, 3, and 4.
But `"yesty"[len("yesty")]` is the same as `"yesty"[5]`, and there is no position 5 in the string.
So we get an `IndexError`, telling us that the positoin we used is not available.

Now let's contrast that with slices.

In [ ]:
print("yesty"[:len("yesty")])

Here we are telling Python to give us the slice from 0 to 5 in `"yesty"`.
Instead of an `IndexError`, we get the entire string.
Whenever the end point of a slice is greater than the last available position, Python just uses the end of the string as the final position.

In [ ]:
print("yesty"[4])
print("yesty"[:4])
print("yesty"[:5])
print("yesty"[:100])

In [ ]:
print("yesty"[2:4])
print("yesty"[2:5])
print("yesty"[2:100])

So why can we go beyond the last position for slices, but not for individual elements?
If you think about how slices are computed, it makes sense that we can get an output even if the end point is beyond the possible range.
We simply build the string as far as possible and then terminate when there are no more positions to look at.
But what is Python to do if we want `"yesty"[5]` and there simply is nothing at this position because the last position is 4?
It could give us the last character `y` as a backup, but this can introduce all kinds of unintended bugs.
Python's philosophy in this case is to say "Dude, I think you made a mistake here, please go ahead and fix it because I don't know what to do with this".
Whereas in the case of slices, there is a safe fallback: "Well, I guess he just wants me to go as far right as possible in this string. Alright, no harm in doing that."

The bottom-line: for individual elements we can only use the available positions, but for slides we can go beyond the right edge of a string without any major problems.

**Exercise.**
In an earlier exercise, you saw that `"yesty"[4:2]` does not return anything.
That is actually not quite true.
It does return something, the *empty string* `""`.
This is a string that contains nothing at all.
It's length is 0, and it contains no positions.

In [ ]:
print("The length of the empty string:")
print(len(""))
print("And its list of positions:")
print(list(enumerate("")))

Using the intuition about how slices work, can you explain why `"yesty"[4:2]` returns a string, but the string it empty?

*put your explanation here*

## Suggesting only common completions

Alright, we now have three different ways of implementing the same idea: iterating over a dictionary with a `for`-loop to find possible completions.
So now we will try to restrict it to the most likely completions.
Note that we are still using a unigram model, so the most likely completions are simply those among the possible completions with the highest frequency.
As we will see, this doesn't work too well in practice, so we'll move to bigrams and trigrams in the next unit to better account for context.

### Getting frequency information

Adding frequency information is fairly easy.
All we need is a bunch of corpora from which we have already constructed counters that tell us for each word type how many tokens it has.
Since we have been doing quite a lot of corpus-based work in the last few units, we have all the code ready for this.

In [ ]:
import urllib.request
import re
from collections import Counter

# we first define custom functions for all individual steps

def get_file(text):
    if text == "hamlet":
        urllib.request.urlretrieve("http://www.gutenberg.org/cache/epub/1524/pg1524.html", "hamlet.txt")
    if text == "faustus":
        urllib.request.urlretrieve("http://www.gutenberg.org/cache/epub/811/pg811.txt", "faustus.txt")
    if text == "johncarter":
        urllib.request.urlretrieve("http://www.gutenberg.org/cache/epub/62/pg62.txt", "johncarter.txt")
        
def read_file(filename):
    with open(filename, "r", encoding="utf-8") as text:
        return text.read()
    
def delete_before_line(string, line):
    return str.split(string, "\n", line)[-1]

def delete_after_line(string, line):
    return str.join("\n", str.split(string, "\n")[:line+1])

def hamlet_cleaner(text):
    # 0. delete unwanted lines
    text = delete_after_line(delete_before_line(text, 366), 10928)
    # 1. remove all headers, i.e. lines starting with <h1, <h2, <h3, and so on
    text = re.sub(r"<h[0-9].*", r"", text)
    # 2. remove speaker information, i.e. lines of the form <p id="id012345789"...<br/>
    text = re.sub(r'<p id="id[0-9]*">[^<]*<br/>', r"", text)
    # 3. remove html tags, i.e. anything of the form <...>
    text = re.sub(r"<[^>]*>", r"", text)
    # 4. remove anything after [ or before ] on a line (this takes care of stage descriptions)
    text = re.sub(r"\[[^\]\n]*", r"", text)
    text = re.sub(r"[^\[\n]*\]", r"", text)
    return text

def faustus_cleaner(text):
    # 0. delete unwanted lines
    text = delete_after_line(delete_before_line(text, 139), 2854)
    # 1. remove stage information
    #    (anything after 10 spaces)
    text = re.sub(r"(\s){10}[^\n]*", r"", text)
    # 2. remove speaker information
    #    (any word in upper caps followed by space or dot)
    text = re.sub(r"[A-Z]{2,}[\s\.]", r"", text)
    # 3. remove anything between square brackets (this takes care of footnote markers)
    text = re.sub(r"\[[^\]]*\]", r"", text)
    return text

def johncarter_cleaner(text):
    # 0. delete unwanted lines
    text = delete_after_line(delete_before_line(text, 234), 6940)
    # 1. delete CHAPTER I
    # (must be done like this because Roman 1 looks like English I)
    text = re.sub("CHAPTER I", "", text)
    # 2. remove any word in upper caps that is longer than 1 character
    text = re.sub(r"[A-Z]{2,}", r"", text)
    # 3. remove anything after [ or before ] on a line
    text = re.sub(r"\[[^\]\n]*", r"", text)
    text = re.sub(r"[^\[\n]*\]", r"", text)
    return text

def tokenize(string):
    return re.findall(r"\w+", string)

def count(token_list):
    return Counter(token_list)


# and now we have two functions that use all the previous functions
# to do all the necessary work for us
def get_and_clean(text):
    get_file(text)
    string = read_file(text + ".txt")
    string = str.lower(string)
    # file-specific cleaning steps
    if text == "hamlet":
        return hamlet_cleaner(string)
    if text == "faustus":
        return faustus_cleaner(string)
    if text == "johncarter":
        return johncarter_cleaner(string)

def tokenize_and_count(string):
    return (count(tokenize(string)))

# and finally we get to run all the code
hamlet = tokenize_and_count(get_and_clean("hamlet"))
faustus = tokenize_and_count(get_and_clean("faustus"))
johncarter = tokenize_and_count(get_and_clean("johncarter"))

So now we can use word counts from *Hamlet*, *Dr. Faustus*, and *The Princess of Mars* to pick out the most frequent word completions.
Note that those are not the most realistic counts for daily usage since nobody writes their emails and text messages like a Victorian play or a pulpy sci-fi novel.
But it's better than nothing - and realistic corpora are rare anyways, many models are still trained on samples from the *Wall Street Journal*, which isn't exactly known for its casual prose.

In addition to those three corpora, we can also treat our dictionary as a corpus where every word type has exactly one token.

In [ ]:
# download the dictionary file
url = "https://raw.githubusercontent.com/dwyl/english-words/master/words.txt"
urllib.request.urlretrieve(url, "words.txt")
dict_string = read_file("words.txt")

# tokenize dict_string;
# remember that each word is on its own line, so [^\n]+ does the trick
dictionary = re.findall(r"[^\n]+", dict_string)

Now we can combine all these counters into a single counter.
Python makes this very easy for us, as we can just use `+` to add two counters.
Adding two counters means the following:

1. If a word type exists in both counters, we add up the two token counts.
1. If a word only exists in one counter, we add it to their combination with its current count.

Check out the cell below for an illustrative toy example.

In [ ]:
counterA = Counter(["a", "a", "a", "b", "b", "c"])
counterB = Counter(["a", "a", "a", "b", "b", "d"])
counterAB = counterA + counterB

print(counterA)
print(counterB)
print(counterAB)

So we can construct a new counter `wordcounts` by turning `dictionary` into a counter and combining it with the counters `hamlet`, `faustus`, and `johncarter`.

In [ ]:
# make dictionary a counter
dict_counter = Counter(dictionary)
# add all the four counters together
wordcounts = dict_counter + hamlet + faustus + johncarter

A word of advice: don't print `wordcounts`, it's going to be one giant counter (remember, `dictionary` contains over 500,000 different words!).
But we can look at the values for some specific words to verify that the combination worked as desired.

In [ ]:
# let's check the values for a few words
for word in ["there", "tender", "IRS", "Microsoft"]:
    print("Showing counts for", word)
    # first the individual counters:
    print("Dictionary:", dict_counter[word])
    print("Hamlet:", hamlet[word])
    print("Faustus:", faustus[word])
    print("Princess of Mars", johncarter[word])
    # and the value for wordcounts should be the sum of the previous four
    print("Total", wordcounts[word])
    # add a new line at the end
    print()

We can also look at the most common words using the `Counter.most_common` function that we encountered in an earlier unit.

In [ ]:
print(Counter.most_common(wordcounts, 50))

So far things have been easy.
Now comes the interesting part: using the counter to get the most likely word completions.
Before proceeding, think a bit about how you would solve the problem, it is a good test of whether you have learned to combine certain general purpose techniques to produce a specialized solution.

Keep thinking...

Keep thinking...

Keep thinking...

Maybe take another few minutes...

Alright, here's how we're gonna do it.
Maybe your solution is the same.
The `Counter.most_common` function tells us the most common words in a counter.
We can use this to get the most likely word completions for any string `foo`:

1. We use the `Counter.copy` function to make a copy `completion_counter` of the `wordcounts` counter.
1. We iterate over the elements of `completion_counter`.
   If a given word `bar` is not a possible completion of `foo`, we set its value to 0: `completion_counter["bar"] = 0`.
   Otherwise we keep the value as is.
1. Once we have checked every word, we use `Counter.most_common(completion_counter, 3)` to get the 3 most likely completions.

**Exercise.**
Implement the procedure above as a custom function `best_completions` that takes two arguments:

1. `string` is the word we want to find the best completions for, and
1. `frequencies` is a counter.

Some of the relevant code has already been added to make your life easier.

In [ ]:
def best_completions(string, frequencies):
    # 1) completion_counter = copy of frequencies
    # 2) iterate over completion_counter
        # 2.1) test if element is a possible completion of string
            # 2.2) if not, set its value to 0
    return Counter.most_common(completion_counter, 3)

Test your code on the examples below.

In [ ]:
print(best_completions("exc", wordcounts))
# intended output: [('except', 29), ('excellent', 19), ('exclaimed', 10)]

In [ ]:
print(best_completions("ther", wordcounts))
# intended output: [('there', 272), ('therefore', 44), ('thereafter', 5)]

In [ ]:
print(best_completions("is", wordcounts))
# intended output: [('is', 751), ('iss', 8), ('issued', 5)]

Alright, this seems to be working.
One minor quibble is that `Counter.most_common` doesn't just give us the word types, but also their counts.
So instead of just `["except", "excellent", "exclaimed"]`, we get `[("except", 29), ("excellent", 19), ("exclaimed", 10)]`.
Elements like `("except", 29)` are called *tuples*.
They are very similar to lists, except that one cannot make any changes to them: items canont be altered, added or removed items.
But we can reference specific positions of a tuple just like we do for strings and lists, and this allows us to write a custom function that removes the counts and only keeps the words.

**Exercise.**
The relevant code is shown below.
Add a comment to each line that explains what it does.

In [ ]:
def strip_counts(most_common):
    words = []
    for entry in most_common:
        word = entry[0]
        list.append(words, word)
    return words

def nice_completions(string, counter):
    return strip_counts(best_completions(string, counter))

In [ ]:
print(nice_completions("exc", wordcounts))

In [ ]:
print(nice_completions("ther", wordcounts))

In [ ]:
print(nice_completions("is", wordcounts))

Good, now we really get a nice list of the most likely completions ordered by frequency in our corpora.
Unfortunately our solution has a severe problem that makes it almost useless in the real world: we are using a unigram model, so we assume that the more frequent the word, the more likely it is to be the desired completion for a word.
But this is not how language works, what words are possible completions depends a lot on the context in the sentence.

For example, suppose the user has already typed *the exc*.
Our model offers three completions for *exc*: *except*, *excellent*, and *exclaimed*.
But two of those cannot be the intended word.
No sentence in English can ever contain the string *the except*, and while *the exclaimed* is possible in phrases like *the exclaimed crossword clue*, it is very unlikely that this is what the user had in mind.
It is much more plausible that the user wants to write *the excellent* or *the exciting* or *the exclusive* or *the excruciating*.
So we cannot simply always suggest the most frequent words.
Instad, we have to suggest the words that are most likely to occur in the current context.
This cannot be done with unigrams; we have to move on to bigrams.